# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [5]:
# Initialize and constants

load_dotenv(override=True)
# api_key = os.getenv('OPENAI_API_KEY')
api_key = os.getenv('GOOGLE_API_KEY')

if api_key and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
# MODEL = 'gpt-5-nano'
MODEL = 'gemini-2.5-flash-lite'
# openai = OpenAI()
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=api_key)

API key looks good so far


In [6]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have Gemini/GPT-5-nano figure out which links are relevant

### Use a call to Gemini/gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [9]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

In [10]:
def select_relevant_links(url):
    response = gemini.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [11]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company profile', 'url': 'https://edwarddonner.com/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/posts/'}]}

In [12]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = gemini.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [13]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gemini-2.5-flash-lite
Found 5 relevant links


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page', 'url': 'https://edwarddonner.com/'},
  {'type': 'company page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'company page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'company page', 'url': 'https://edwarddonner.com/posts/'}]}

In [14]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gemini-2.5-flash-lite
Found 9 relevant links


{'links': [{'type': 'models', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces', 'url': 'https://huggingface.co/spaces'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs'},
  {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'about page', 'url': 'https://huggingface.co/learn'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [15]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [16]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gemini-2.5-flash-lite
Found 10 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Tongyi-MAI/Z-Image-Turbo
Updated
about 21 hours ago
•
217k
•
2.39k
microsoft/VibeVoice-Realtime-0.5B
Updated
about 9 hours ago
•
57k
•
569
deepseek-ai/DeepSeek-V3.2
Updated
8 days ago
•
33.1k
•
832
alibaba-pai/Z-Image-Turbo-Fun-Controlnet-Union
Updated
7 days ago
•
285
zai-org/GLM-4.6V-Flash
Updated
about 8 hours ago
•
6.92k
•
200
Browse 1M+ models
Spaces
Running
on
Zero
MCP
Featured
1.26k
Z Image Turbo
🏃
1.26k
Generate images from text prompts
Running
on
Zero
426
Z Image Turbo
🖼
426
Generate stunning images from text prom

In [17]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gemini-2.5-flash-lite
Found 11 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nTongyi-MAI/Z-Image-Turbo\nUpdated\nabout 21 hours ago\n•\n217k\n•\n2.39k\nmicrosoft/VibeVoice-Realtime-0.5B\nUpdated\nabout 9 hours ago\n•\n57k\n•\n569\ndeepseek-ai/DeepSeek-V3.2\nUpdated\n8 days ago\n•\n33.1k\n•\n832\nalibaba-pai/Z-Image-Turbo-Fun-Controlnet-Union\nUpdated\n7 days ago\n•\n285\nzai-org/GLM-4.6V-Flash\nUpdated\nabout 8 hours ago\n•\n6.92k\n•\n200\nBrowse 1M+ models\nSpaces\n

In [20]:
def create_brochure(company_name, url):
    response = gemini.chat.completions.create(
        # model="gpt-4.1-mini",
        model='gemini-2.5-flash-lite',
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [21]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gemini-2.5-flash-lite
Found 10 relevant links


# Hugging Face: The AI Community Building the Future

Hugging Face is the premier platform for the machine learning community to collaborate, discover, and build the future of AI. We are the home of open-source AI, offering a collaborative space for an ever-growing community to share and advance AI technologies.

## What We Offer

*   **1M+ Models:** Explore a vast and ever-expanding collection of machine learning models covering diverse tasks and modalities, including text, image, video, and audio.
*   **250k+ Datasets:** Access a rich repository of datasets to train and evaluate your AI models.
*   **400k+ Applications (Spaces):** Discover and run cutting-edge AI applications, from image generation to LLM evaluation tools.

## The Hugging Face Platform

Our platform empowers you to:

*   **Collaborate:** Host and work on unlimited public models, datasets, and applications.
*   **Accelerate Your ML:** Leverage our open-source stack and paid compute solutions to move faster.
*   **Explore All Modalities:** Engage with AI across text, image, video, audio, and even 3D.
*   **Build Your Portfolio:** Showcase your work to the world and build your machine learning profile.

## For Our Community

Hugging Face is more than just a platform; it's a vibrant community. We foster an environment where developers, researchers, and enthusiasts can connect, share knowledge, and push the boundaries of artificial intelligence together.

## For Businesses

We offer Enterprise solutions designed to help organizations accelerate their AI initiatives with enhanced security, support, and scalability.

## Join Us

Whether you're looking to contribute to the open-source AI movement, build your next AI application, or explore the latest in machine learning, Hugging Face is the place to be.

**Sign Up today and become part of the AI community building the future!**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [22]:
def stream_brochure(company_name, url):
    stream = gemini.chat.completions.create(
        # model="gpt-4.1-mini",
        model='gemini-2.5-flash-lite',
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gemini-2.5-flash-lite
Found 11 relevant links


# Hugging Face: The AI Community Building the Future

Hugging Face is the leading platform where the machine learning community collaborates on models, datasets, and applications. We are building the future of AI, together.

## What We Offer:

*   **1M+ Models:** Explore a vast and ever-growing repository of cutting-edge machine learning models. Whether you're working with text, image, video, audio, or 3D data, you'll find the tools you need.
*   **400k+ Applications (Spaces):** Discover and deploy a wide array of AI applications, from image and video generation to LLM evaluation tools. Many are powered by our efficient inference solutions.
*   **250k+ Datasets:** Access a diverse collection of datasets to train and evaluate your AI models, covering a wide range of tasks and domains.

## The Hugging Face Platform:

*   **Collaborate Seamlessly:** Host and work on unlimited public models, datasets, and applications in a collaborative environment.
*   **Move Faster:** Leverage the power of the HF Open Source stack to accelerate your machine learning development.
*   **Build Your Portfolio:** Showcase your work to the world and build your Machine Learning profile by sharing your contributions.

## For Businesses:

Accelerate your AI initiatives with our paid Compute and Enterprise solutions, designed to provide enhanced performance and support.

## Join the Community:

Hugging Face is more than just a platform; it's a vibrant community dedicated to advancing AI. Connect with like-minded individuals, share your knowledge, and be a part of the next AI revolution.

**Sign Up Today and start creating, discovering, and collaborating on ML!**

In [25]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gemini-2.5-flash-lite
Found 12 relevant links


**Hugging Face: Where AI Gets a Hug (and So Do You!)**

Tired of AI that feels cold and distant? Yearning for a place where algorithms meet amicability? Welcome to Hugging Face, the digital watering hole for all things Machine Learning! We're not just building the future; we're building it with a smile, a community, and a whole lot of open-source magic.

**What in the World is Hugging Face?**

Imagine a bustling digital city square, but instead of pigeons and street performers, you've got over a million models, datasets, and applications waiting to be discovered. That's the Hugging Face Hub! It's the ultimate playground for ML enthusiasts, scientists, and anyone curious enough to ask "what if?" We're the collaboration platform that's democratizing AI, making it accessible, shareable, and dare we say, fun.

**Our Vibe? Think Golden Retriever, But for Code.**

We're a community. We're open-source. We're ethical (we promise!). Our brand colors are cheerful yellow and energetic orange, which pretty much sums up our attitude. We believe in sharing, learning, and building awesome stuff together. If you're a brilliant mind looking to make a real impact, you'll fit right in. We're at the heart of the AI revolution, and we like to think we bring the party.

**Who Uses Our Stuff? Pretty Much Everyone Awesome.**

From researchers pushing the boundaries of LLMs to developers creating mind-blowing image generators (seriously, check out "Z Image Turbo"!), our platform is where innovation happens. We've got datasets for every whim, models for every task, and applications that will make you say "wow" (or at least "ooh, shiny!").

**Fancy a Gig at the AI Playground?**

Dreaming of a career where you can experiment with cutting-edge AI, collaborate with brilliant minds, and contribute to an open and ethical future? Then you, my friend, might be a Hugger Face in the making! We're always on the lookout for talented individuals to join our growing team. Check out our careers page – you might just find your next adventure.

**Ready to Dive In?**

*   **Explore 1M+ Models:** Go ahead, browse. We dare you.
*   **Discover 400k+ Apps:** See what others are building. Prepare to be inspired (and maybe a little jealous).
*   **Unearth 250k+ Datasets:** Because every great AI needs great data.
*   **Learn with Us:** From LLMs to Robotics to Diffusion, we've got courses to turn you into an AI wizard.

**Hugging Face: Come for the AI, Stay for the Community (and the Hugs).**

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>